# 実装説明

1 iterのpruningを通して実装を理解します

- 元論文 [1611.06440 Pruning Convolutional Neural Networks for Resource Efficient Inference](https://arxiv.org/abs/1611.06440)
- [pytorchでの実装](https://github.com/jacobgil/pytorch-pruning)
- [実装者によるブログ記事](https://jacobgil.github.io/deeplearning/pruning-deep-learning)


実装のキモは以下の2点です
- 各フィルタの重要度の計算
- 重要度が最も低いn枚のフィルタを削除した新たなネットワークの構築

## ライブラリのインポート

In [1]:
import torch
from torch.autograd import Variable
from torchvision import models
import cv2
import sys
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dataset
from prune import *
import argparse
from operator import itemgetter
from heapq import nsmallest
import time

## pruningしたい学習済みモデルの読み込み

In [2]:
class ModifiedVGG16Model(torch.nn.Module):
    def __init__(self):
        super(ModifiedVGG16Model, self).__init__()

        model = models.vgg16(pretrained=True)
        self.features = model.features

        for param in self.features.parameters():
            param.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(25088, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = torch.load("model").cuda()

TypeError: <module '__main__'> is a built-in class

### ※上記でエラーが起こる場合，別のファイルからクラスをインポートする

In [3]:
from finetune import ModifiedVGG16Model
model = torch.load("model").cuda()

## データセットの準備

本実装では，フォーク元(https://jacobgil.github.io/deeplearning/pruning-deep-learning) と同様に以下のデータを使用します。
- 犬猫の2クラス分類のデータセット(https://www.kaggle.com/c/dogs-vs-cats)
- トレーニングデータ1000枚ずつ，テストデータ400枚ずつ

In [4]:
train_path = "train"
test_path = "test"

train_data_loader = dataset.loader(train_path)
test_data_loader = dataset.test_loader(test_path)

#model.train()

## 各フィルタの重要度を計算

各フィルタの重要度`grad*activation`を計算し、辞書`filter_ranks`に保存します。

`register_hook`を用いて、backwardの際に重要度を計算するようにします。

### 勾配と特徴マップからフィルタの重要度を返す関数
`register_hook`に渡す関数は以下です。（解説は後述）

In [5]:
# function for register_hook
# compute grad*activation
def compute_rank(grad):
    global grad_index
    global activations
    global filter_ranks
    activation_index = len(activations) - grad_index - 1
    activation = activations[activation_index]

    values = torch.sum((activation * grad), dim = 0, keepdim=True).sum(dim=2, keepdim=True).sum(dim=3, keepdim=True)[0, :, 0, 0].data

    # Normalize the rank by the filter dimensions
    values = values / (activation.size(0) * activation.size(2) * activation.size(3))

    if activation_index not in filter_ranks:
        filter_ranks[activation_index] = torch.FloatTensor(activation.size(1)).zero_().cuda()

    filter_ranks[activation_index] += values
    grad_index += 1

### フィルタの重要度を計算するループ

フィルタの重要度を計算します。
- 順伝播の際に`activation`を記録、各層の出力`x`に`compute_rank`のフックを登録
- 逆伝播の際に`compute_rank`が呼び出され`grad*activation`を計算、`filter_ranks`に保存

In [6]:
criterion = torch.nn.CrossEntropyLoss()

# dictonary storing importances of each filter
filter_ranks = {}

# back propagation loop computing importances
for batch, label in train_data_loader:
    model.zero_grad()
    x = Variable(batch.cuda(), requires_grad = True)

    activations = []
    grad_index = 0
    activation_to_layer = {}
    activation_index = 0

    # forward
    for layer, (name, module) in enumerate(model.features._modules.items()):
        # print("layer, name, module: ", layer, name, module)

        x = module(x)
        
        # store activation
        if isinstance(module, torch.nn.modules.conv.Conv2d):
            x.register_hook(compute_rank)
            activations.append(x)
            activation_to_layer[activation_index] = layer
            activation_index += 1

    # print(x.data.cpu().numpy().shape)
    out = model.classifier(x.view(x.size(0), -1))

    # backward, compute compute_rank()
    # no update of weights
    criterion(out, Variable(label.cuda())).backward()

### 少し詳しい説明
上記のソースをもう少し詳しく解説します。

順伝播では、元々のモデルのソースに変更を加えたくないため、`model.features._modules.items()`でモデルの各層を関数`module`として使用します。

In [8]:
# forward
for layer, (name, module) in enumerate(model.features._modules.items()):
    print("layer, name, module: ", layer, name, module)

layer, name, module:  0 0 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer, name, module:  1 1 ReLU (inplace)
layer, name, module:  2 2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer, name, module:  3 3 ReLU (inplace)
layer, name, module:  4 4 MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
layer, name, module:  5 5 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer, name, module:  6 6 ReLU (inplace)
layer, name, module:  7 7 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer, name, module:  8 8 ReLU (inplace)
layer, name, module:  9 9 MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
layer, name, module:  10 10 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer, name, module:  11 11 ReLU (inplace)
layer, name, module:  12 12 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
layer, name, module:  13 13 ReLU (inplace)
layer, name, module:  14

#### filter_ranksの中身を確認する
`filter_ranks`は、各CONV層におけるフィルタの重要度を保存した辞書です。

キーにはCONV層の番号(全体の層番号ではなく、何番目のCONV層か)、バリューには各層のフィルタ数と同サイズの配列が格納されます。たとえば以下を実行すると、2番目（3つ目）のCONV層の10番目のフィルタの重要度が確認できます。

In [9]:
print(filter_ranks[2][10])

6.786516593138003e-08


全体を確認します。

In [10]:
for i in filter_ranks:
    f = filter_ranks[i].cpu().numpy()
    print(i, f.shape, f[:10])

0 (64,) [ -3.73500662e-08  -2.04436663e-08  -4.24422844e-08   1.50498369e-09
  -1.22746599e-07   1.56757203e-08  -7.06289285e-08  -1.56441722e-08
   4.98041928e-08   6.61597568e-08]
1 (64,) [ -6.03485262e-08   1.54667390e-08  -1.66291812e-07  -1.09614007e-09
  -4.62700172e-08  -3.33858274e-08  -1.93150616e-08   1.03554010e-08
  -5.13008942e-08  -4.12457482e-08]
2 (128,) [  7.49703801e-08   1.48712473e-07  -2.28549837e-07  -1.70827221e-07
  -3.12936493e-07  -2.63289213e-08  -2.85883104e-08  -1.14331733e-07
  -2.27099548e-07  -3.41180915e-08]
3 (128,) [  5.78968269e-08  -3.95947012e-07  -3.97092407e-07   3.17554111e-07
   7.05701497e-09   2.53987679e-07  -2.54641233e-08   5.12535685e-08
  -3.63208699e-08  -1.13324226e-07]
4 (256,) [  8.22116363e-07  -1.80695736e-07  -2.16293529e-07   1.20318447e-07
  -2.78650667e-08   3.89945178e-08  -2.46329193e-07   2.94861366e-07
  -3.94033407e-07  -3.23816550e-07]
5 (256,) [ -1.88000655e-07  -7.05653406e-07  -1.78253941e-07  -2.87118553e-07
   1.4767

### compute_rankの動作内容を確認する

`compute_rank`の動作を説明します。

まず、順伝播によって計算されたある層のactivationを取得します。ここでは第2層とします。

In [11]:
grad = 0.01
activation_index = 2
activation = activations[activation_index]
print(activations[2].data.cpu().numpy().shape)

(16, 128, 112, 112)


逆伝播によって計算された勾配`grad`と各`activation`の要素積を取ります。
ここでは簡単のため、`grad`の値は全て`0.0001`とします。

In [12]:
grad = 0.0001

product = activation * grad
print(product.data.shape)

torch.Size([16, 128, 112, 112])


チャンネル毎に要素を足し合わせます。各チャンネルに重要度の値がひとつずつある状態になりました。

In [13]:
values = torch.sum(product, dim = 0, keepdim=True).sum(dim=2, keepdim=True).sum(dim=3, keepdim=True)[0, :, 0, 0].data

print(values[:10])
print(values.shape)


  3.9450
 12.9920
 15.2859
 15.6443
 16.1155
  9.4931
  6.8969
 13.4246
 12.3292
  9.0060
[torch.cuda.FloatTensor of size 10 (GPU 0)]

torch.Size([128])


足し合わせた数だけ値を割ります。

In [14]:
# Normalize the rank by the filter dimensions
values = values / (activation.size(0) * activation.size(2) * activation.size(3))

print(values[:10])
print(values.shape)


1.00000e-05 *
  1.9656
  6.4732
  7.6162
  7.7947
  8.0295
  4.7299
  3.4363
  6.6887
  6.1430
  4.4872
[torch.cuda.FloatTensor of size 10 (GPU 0)]

torch.Size([128])


このベクトルを、`filter_ranks[2]`のバリューに足し合わせます。
ここでは、先程の計算結果に影響が出ないようダミーの`filter_ranks`を用います。
以上がcompute_rankの動作説明です。

In [15]:
dummy_filter_ranks = {}

if activation_index not in dummy_filter_ranks:
    dummy_filter_ranks[activation_index] = torch.FloatTensor(activation.size(1)).zero_().cuda()

dummy_filter_ranks[activation_index] += values

print(dummy_filter_ranks)

{2: 
1.00000e-04 *
  0.1966
  0.6473
  0.7616
  0.7795
  0.8029
  0.4730
  0.3436
  0.6689
  0.6143
  0.4487
  0.5297
  0.9691
  0.8908
  0.4443
  0.9500
  0.6043
  0.4370
  0.8381
  0.7852
  0.8091
  0.3340
  0.7913
  0.7001
  0.7547
  0.5177
  0.8574
  0.9816
  0.8555
  0.0756
  0.4242
  1.0942
  1.0451
  0.6496
  0.8781
  0.6409
  1.2347
  0.9718
  1.0940
  0.5992
  0.3715
  0.8031
  0.4157
  0.8682
  0.9762
  0.6648
  0.5830
  0.3536
  0.9903
  0.5256
  0.8474
  0.6614
  0.5325
  0.7867
  0.9657
  0.4549
  0.9072
  0.1479
  0.9983
  0.6620
  0.7351
  0.4536
  0.7857
  0.5795
  0.7157
  0.1190
  0.7701
  0.4999
  0.4382
  0.4138
  0.2965
  1.0581
  0.7284
  0.4779
  0.5061
  0.4166
  0.2027
  1.1202
  0.9134
  0.6500
  0.9310
  0.1543
  0.6064
  0.4601
  0.8077
  0.4324
  0.6722
  0.1460
  0.6274
  0.3124
  0.4643
  0.7515
  0.0745
  0.9471
  0.8134
  0.3300
  0.5231
  0.7472
  0.4664
  0.9025
  0.3407
  0.3941
  0.3472
  0.2645
  0.4397
  0.4302
  0.6278
  0.7422
  1.3372
  0.4326


## 重要度の順位付け、pruning対象となるフィルタの決定

以上で計算した重要度を元に、重要度の最も低いフィルタをpruning対象とします。

### 正規化

`filter_ranks`のスケールが層によって異なるため、層ごとにL2正規化を行います。

In [16]:
# before normalization
for i in filter_ranks:
    f = filter_ranks[i].cpu().numpy()
    print(i, f.shape, f[:5])

# normalize_ranks_per_layer()
for i in filter_ranks:
    v = torch.abs(filter_ranks[i])
    v = v / np.sqrt(torch.sum(v * v))
    filter_ranks[i] = v.cpu()

# after normalization
for i in filter_ranks:
    f = filter_ranks[i].cpu().numpy()
    print(i, f.shape, f[:3])

0 (64,) [ -3.73500662e-08  -2.04436663e-08  -4.24422844e-08   1.50498369e-09
  -1.22746599e-07]
1 (64,) [ -6.03485262e-08   1.54667390e-08  -1.66291812e-07  -1.09614007e-09
  -4.62700172e-08]
2 (128,) [  7.49703801e-08   1.48712473e-07  -2.28549837e-07  -1.70827221e-07
  -3.12936493e-07]
3 (128,) [  5.78968269e-08  -3.95947012e-07  -3.97092407e-07   3.17554111e-07
   7.05701497e-09]
4 (256,) [  8.22116363e-07  -1.80695736e-07  -2.16293529e-07   1.20318447e-07
  -2.78650667e-08]
5 (256,) [ -1.88000655e-07  -7.05653406e-07  -1.78253941e-07  -2.87118553e-07
   1.47676253e-08]
6 (256,) [ -1.84796690e-07  -2.70328258e-07  -2.81789852e-07  -1.09626865e-06
  -3.09229421e-07]
7 (512,) [  6.66967253e-07  -7.56142185e-07  -1.35905026e-07  -1.18118214e-06
  -5.31615513e-07]
8 (512,) [  4.30816698e-07  -1.79872245e-07  -4.61390755e-06  -1.18717526e-06
   5.99207453e-07]
9 (512,) [ -3.60778495e-06  -5.40017879e-07   1.03791353e-06  -3.20131051e-07
  -1.15264629e-06]
10 (512,) [  1.43056968e-06   3.

### 重要度の最も小さいフィルタをn枚抽出する

全ての層から、重要度の最も小さいフィルタを集めて以下の形式のリストにします。
- (層番号，フィルタ番号，フィルタの重要度)

In [17]:
# number of filters prunned
num = 256

data = []
for i in sorted(filter_ranks.keys()):
    for j in range(filter_ranks[i].size(0)):
        data.append((activation_to_layer[i], j, filter_ranks[i][j]))

filters_to_prune = nsmallest(num, data, itemgetter(2))

print(filters_to_prune.__class__)
print(len(filters_to_prune))
for i in range(5):
    print(filters_to_prune[i])

<class 'list'>
256
(21, 40, 1.0759843682706105e-08)
(17, 78, 5.7456950344203506e-06)
(24, 161, 6.677765668428037e-06)
(28, 258, 1.5789490134920925e-05)
(24, 137, 2.9008037017774768e-05)


### heapq.nsmallest(n, iterable, key=None)

https://docs.python.jp/3/library/heapq.html

>iterable で定義されるデータセットのうち、最小値から昇順に n 個の値のリストを返します。
>(あたえられた場合) key は、引数を一つとる、iterable のそれぞれの要素から比較キーを生成する関数を指定します: key=str.lower 以下のコードと同等です: sorted(iterable, key=key)[:n]

### key: 全体での層番号、value: pruning対象のフィルタ番号の辞書を作成

In [18]:
filters_to_prune_per_layer = {}
for (l, f, _) in filters_to_prune:
    if l not in filters_to_prune_per_layer:
        filters_to_prune_per_layer[l] = []
    filters_to_prune_per_layer[l].append(f)

for i in sorted(filters_to_prune_per_layer.items()):
    print("layer", i[0],   ": ", len(i[1]), "filters, ", i[1])

layer 0 :  1 filters,  [45]
layer 2 :  1 filters,  [3]
layer 5 :  1 filters,  [70]
layer 7 :  4 filters,  [99, 88, 22, 14]
layer 10 :  11 filters,  [128, 209, 138, 28, 197, 123, 236, 169, 119, 87, 52]
layer 12 :  7 filters,  [235, 181, 118, 105, 23, 27, 80]
layer 14 :  9 filters,  [233, 41, 185, 248, 111, 168, 25, 255, 18]
layer 17 :  24 filters,  [78, 457, 277, 301, 502, 308, 181, 318, 102, 136, 418, 270, 280, 267, 461, 266, 53, 192, 295, 463, 432, 323, 419, 333]
layer 19 :  21 filters,  [130, 222, 347, 377, 118, 380, 122, 7, 293, 73, 438, 353, 124, 382, 306, 11, 83, 114, 419, 92, 64]
layer 21 :  20 filters,  [40, 22, 290, 112, 75, 175, 212, 131, 176, 111, 361, 291, 437, 504, 510, 276, 299, 115, 419, 355]
layer 24 :  41 filters,  [161, 137, 276, 333, 211, 192, 219, 467, 500, 393, 203, 28, 317, 438, 490, 248, 135, 351, 99, 205, 160, 101, 238, 230, 171, 66, 95, 97, 156, 222, 167, 29, 433, 272, 266, 252, 408, 322, 382, 198, 414]
layer 26 :  41 filters,  [53, 239, 33, 238, 77, 335, 404, 3

### 削除後のインデックスを考慮して、フィルタ番号を振り直す

例）フィルタ削除の順番が「0->24->30」だったとします。
0番を削除した場合、第24番、第30番のフィルタはそれぞれ第23番、第29番になります。
次に第23番のフィルタを削除した場合、第29番のフィルタは第28番になります。
よって，番号を0, 23, 28と振り直します。

In [19]:
for l in filters_to_prune_per_layer:
    filters_to_prune_per_layer[l] = sorted(filters_to_prune_per_layer[l])
    for i in range(len(filters_to_prune_per_layer[l])):
        filters_to_prune_per_layer[l][i] = filters_to_prune_per_layer[l][i] - i

for i in sorted(filters_to_prune_per_layer.items()):
    print("layer", i[0],   ": ", len(i[1]), "filters, ", i[1])

layer 0 :  1 filters,  [45]
layer 2 :  1 filters,  [3]
layer 5 :  1 filters,  [70]
layer 7 :  4 filters,  [14, 21, 86, 96]
layer 10 :  11 filters,  [28, 51, 85, 116, 119, 123, 132, 162, 189, 200, 226]
layer 12 :  7 filters,  [23, 26, 78, 102, 114, 176, 229]
layer 14 :  9 filters,  [18, 24, 39, 108, 164, 180, 227, 241, 247]
layer 17 :  24 filters,  [53, 77, 100, 133, 177, 187, 260, 260, 262, 268, 270, 284, 289, 295, 304, 308, 317, 401, 401, 413, 437, 440, 441, 479]
layer 19 :  21 filters,  [7, 10, 62, 70, 79, 87, 108, 111, 114, 115, 120, 211, 281, 293, 333, 338, 361, 363, 364, 400, 418]
layer 21 :  20 filters,  [22, 39, 73, 108, 108, 110, 125, 168, 168, 203, 266, 279, 279, 286, 341, 346, 403, 420, 486, 491]
layer 24 :  41 filters,  [28, 28, 64, 92, 93, 94, 95, 128, 129, 147, 150, 150, 155, 158, 178, 183, 187, 188, 193, 200, 202, 209, 216, 225, 228, 241, 246, 249, 289, 293, 303, 320, 350, 360, 374, 379, 397, 401, 429, 451, 460]
layer 26 :  41 filters,  [26, 30, 31, 50, 68, 72, 76, 99, 11

### (層番号, フィルタ番号)のリストに直す
これでpruningの準備は完了です。

In [20]:
prune_targets = []
for l in filters_to_prune_per_layer:
    for i in filters_to_prune_per_layer[l]:
        prune_targets.append((l, i))

print(len(prune_targets))
prune_targets

256


[(0, 45),
 (2, 3),
 (5, 70),
 (7, 14),
 (7, 21),
 (7, 86),
 (7, 96),
 (10, 28),
 (10, 51),
 (10, 85),
 (10, 116),
 (10, 119),
 (10, 123),
 (10, 132),
 (10, 162),
 (10, 189),
 (10, 200),
 (10, 226),
 (12, 23),
 (12, 26),
 (12, 78),
 (12, 102),
 (12, 114),
 (12, 176),
 (12, 229),
 (14, 18),
 (14, 24),
 (14, 39),
 (14, 108),
 (14, 164),
 (14, 180),
 (14, 227),
 (14, 241),
 (14, 247),
 (17, 53),
 (17, 77),
 (17, 100),
 (17, 133),
 (17, 177),
 (17, 187),
 (17, 260),
 (17, 260),
 (17, 262),
 (17, 268),
 (17, 270),
 (17, 284),
 (17, 289),
 (17, 295),
 (17, 304),
 (17, 308),
 (17, 317),
 (17, 401),
 (17, 401),
 (17, 413),
 (17, 437),
 (17, 440),
 (17, 441),
 (17, 479),
 (19, 7),
 (19, 10),
 (19, 62),
 (19, 70),
 (19, 79),
 (19, 87),
 (19, 108),
 (19, 111),
 (19, 114),
 (19, 115),
 (19, 120),
 (19, 211),
 (19, 281),
 (19, 293),
 (19, 333),
 (19, 338),
 (19, 361),
 (19, 363),
 (19, 364),
 (19, 400),
 (19, 418),
 (21, 22),
 (21, 39),
 (21, 73),
 (21, 108),
 (21, 108),
 (21, 110),
 (21, 125),
 (21

## フィルタをpruningして新たなモデルを作成する

`prune.py`の`prune_vgg16_conv_layer`は、モデルからフィルタを1枚削除した新しいモデルを返す関数です。
これをfor分で上記で指定したpruning枚数分(ここでは256枚分)繰り返します。

In [21]:
layers_prunned = {}
for layer_index, filter_index in prune_targets:
    if layer_index not in layers_prunned:
        layers_prunned[layer_index] = 0
    layers_prunned[layer_index] = layers_prunned[layer_index] + 1 

print("Layers that will be prunned", layers_prunned)
print("Prunning filters.. ")
prunned_model = model.cpu()

for layer_index, filter_index in prune_targets:
    print(layer_index, filter_index)
    prunned_model = prune_vgg16_conv_layer(model, layer_index, filter_index)

Layers that will be prunned {0: 1, 2: 1, 5: 1, 7: 4, 10: 11, 12: 7, 14: 9, 17: 24, 19: 21, 21: 20, 24: 41, 26: 41, 28: 75}
Prunning filters.. 
0 45
2 3
5 70
7 14
7 21
7 86
7 96
10 28
10 51
10 85
10 116
10 119
10 123
10 132
10 162
10 189
10 200
10 226
12 23
12 26
12 78
12 102
12 114
12 176
12 229
14 18
14 24
14 39
14 108
14 164
14 180
14 227
14 241
14 247
17 53
17 77
17 100
17 133
17 177
17 187
17 260
17 260
17 262
17 268
17 270
17 284
17 289
17 295
17 304
17 308
17 317
17 401
17 401
17 413
17 437
17 440
17 441
17 479
19 7
19 10
19 62
19 70
19 79
19 87
19 108
19 111
19 114
19 115
19 120
19 211
19 281
19 293
19 333
19 338
19 361
19 363
19 364
19 400
19 418
21 22
21 39
21 73
21 108
21 108
21 110
21 125
21 168
21 168
21 203
21 266
21 279
21 279
21 286
21 341
21 346
21 403
21 420
21 486
21 491
24 28
24 28
24 64
24 92
24 93
24 94
24 95
24 128
24 129
24 147
24 150
24 150
24 155
24 158
24 178
24 183
24 187
24 188
24 193
24 200
24 202
24 209
24 216
24 225
24 228
24 241
24 246
24 249
24 289
24 2

### pruning前後のモデルのチャンネル数を比較する

#### pruning前

In [69]:
before = model.features._modules
after = prunned_model.features._modules
for i in range(len(before)):
    print(i, before[str(i)])
    print(i, after[str(i)])

NameError: name 'prunned_model' is not defined

### 推論テスト
最後に、pruning前後での推論の精度と実行時間を比較します。
今回はfine-tuningを行っていないため、pruning後は精度が落ちます。

In [17]:
# before pruning
model = model.cuda()

model.eval()
correct = 0
total = 0

start = time.time()


for i, (batch, label) in enumerate(test_data_loader):
    batch = batch.cuda()
    output = model(Variable(batch))
    pred = output.data.max(1)[1]
    correct += pred.cpu().eq(label).sum()
    total += label.size(0)

print("Accuracy :", float(correct) / total)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

Accuracy : 0.94125


ModifiedVGG16Model (
  (features): Sequential (
    (0): Conv2d(3, 61, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU (inplace)
    (2): Conv2d(61, 61, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU (inplace)
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d(61, 124, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU (inplace)
    (7): Conv2d(124, 124, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU (inplace)
    (9): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d(124, 242, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): Conv2d(242, 249, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU (inplace)
    (14): Conv2d(249, 251, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU (inplace)
    (16): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d(251, 481, kernel_size=(3, 3), st

In [ ]:
# after pruning
model = prunned_model.cuda()

model.eval()
correct = 0
total = 0

start = time.time()


for i, (batch, label) in enumerate(test_data_loader):
    batch = batch.cuda()
    output = model(Variable(batch))
    pred = output.data.max(1)[1]
    correct += pred.cpu().eq(label).sum()
    total += label.size(0)

print("Accuracy :", float(correct) / total)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")